In [ ]:
from scipy.io import loadmat
import pandas as pd

data = loadmat('./test/hokuto_test1.mat')
print(data)

In [2]:
import pandas as pd
import numpy as np

# Assuming 'data' is your dictionary containing the MATLAB file content
data_dict = data

# Extract the sensor data; this path might need to be adjusted based on your data's full structure
sensor_data = data_dict['D'][0]

# Initialize lists to hold extracted information
labels = []
types = []
units = []
measurements_x = []
measurements_y = []

# Iterate through each sensor's data
for sensor in sensor_data:
    labels.append(sensor[1][0])
    types.append(sensor[2][0])
    units.append(sensor[5][0])
    measurements_x.append(sensor[3][0] if sensor[3].size else np.nan)
    measurements_y.append(sensor[4][0] if sensor[4].size else np.nan)

# Create a DataFrame
df = pd.DataFrame({
    'Label': labels,
    'Type': types,
    'Unit': units,
    'Measurement_X': measurements_x,
    'Measurement_Y': measurements_y,
})

df


NameError: name 'data' is not defined

In [3]:
import pandas as pd
import numpy as np

# Information basée sur votre structure de données
data_path = 'traininig/mci/hokuto_mci1.dat'  # Chemin vers le fichier .dat
# Exemple de dimensions [nombre de canaux, nombre d'échantillons]
dim = [224, 300000]

# Ce script suppose que le fichier .dat peut être chargé comme un tableau NumPy directement
# Vous devrez peut-être ajuster cela en fonction du format réel du fichier .dat

# Fonction pour charger les données MEG depuis un fichier .dat


def load_meg_data(filepath, dimensions):
    # Charger le fichier .dat comme un tableau NumPy
    # La manière exacte de charger dépend du format des données dans le fichier .dat
    data = np.memmap(filepath, dtype=np.float32,
                     mode='r', shape=tuple(dimensions))
    print(data)
    # Convertir en DataFrame
    df = pd.DataFrame(data)
    return df

# Tentative de chargement des données (ne fonctionnera pas dans cet environnement)
df = load_meg_data(data_path, dim)

#df.describe()

FileNotFoundError: [Errno 2] No such file or directory: 'traininig/mci/hokuto_mci1.dat'

In [ ]:
df.head()

In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import rfft, rfftfreq

# Supposons que 'df' est votre DataFrame contenant les données MEG


def alpha_power(data, fs):
    alpha_powers = []
    for index, row in data.iterrows():
        # S'assurer que la ligne est convertie en un tableau NumPy unidimensionnel
        signal = np.array(row, dtype=np.float64)

        # Appliquer la FFT
        fft_vals = rfft(signal)
        fft_freq = rfftfreq(len(signal), 1/fs)

        # Filtrer pour obtenir la bande alpha (8-12 Hz)
        alpha_indices = (fft_freq >= 8) & (fft_freq <= 12)
        alpha_fft = fft_vals[alpha_indices]

        # Calculer la puissance moyenne dans la bande alpha
        alpha_power = np.mean(np.abs(alpha_fft)**2)
        alpha_powers.append(alpha_power)

    return np.mean(alpha_powers)


# Exemple de fréquence d'échantillonnage
fs = 1000  # Hz

# Calculer la puissance moyenne dans la bande alpha pour le DataFrame 'df'
mean_alpha_power = alpha_power(df, fs)

# Afficher la puissance moyenne dans la bande alpha
print("Puissance moyenne dans la bande alpha :", mean_alpha_power)

In [ ]:
import mne
import os
from pathlib import Path

# Définition des chemins de base pour les données Fieldtrip et pour les sorties en format .fif
# Chemin de base modifié pour correspondre à votre structure de dossier
base_path = Path("./")
path_fieldtrip = base_path / "training"
path_fif = base_path / "converted_fif"

# Création du répertoire de sortie s'il n'existe pas
path_fif.mkdir(parents=True, exist_ok=True)


def convert_to_fif(input_path, output_path):
    """
    Convertit les fichiers Fieldtrip en format .fif en utilisant MNE-Python.
    """
    # Création des dossiers de sortie s'ils n'existent pas
    (output_path / "control").mkdir(parents=True, exist_ok=True)
    (output_path / "mci").mkdir(parents=True, exist_ok=True)
    (output_path / "dementia").mkdir(parents=True, exist_ok=True)

    for subdir, dirs, files in os.walk(str(input_path)):
        for file in files:
            if file.endswith(".mat"):  # Assurez-vous que le fichier est un fichier MATLAB
                print('yes')
                # Lecture du fichier Fieldtrip
                raw_fieldtrip = mne.io.read_raw_f   ieldtrip(
                    Path(subdir) / file, None, data_name='D')
                # Conversion du fichier en format .fif
                raw_fif = create_raw(raw_fieldtrip)
                # Construction du chemin de sortie
                rel_path = Path(subdir).relative_to(input_path)
                fif_path = output_path / rel_path / \
                    file.replace(".mat", ".fif")
                # Sauvegarde du fichier converti
                raw_fif.save(fif_path)


def create_raw(raw_fieldtrip):
    """
    Crée un objet Raw à partir de données Fieldtrip chargées.
    """
    raw_copy = raw_fieldtrip.pick_types(meg=True)
    raw_copy.load_data()

    data = raw_copy.get_data()
    data *= 1e-15  # Conversion de fT à T

    ch_types = raw_copy.get_channel_types()
    ch_names = raw_copy.ch_names
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=1000)

    raw_fif = mne.io.RawArray(data, info)
    return raw_fif


# Lancer la conversion
print(path_fieldtrip)
convert_to_fif(path_fieldtrip, path_fif)

In [1]:
import mne
import os
from pathlib import Path

# Chemins en dur où se trouvent les données
path_fieldtrip = Path("./training")
path_fif = Path("./training_fif")

# Création des dossiers de destination pour les fichiers .fif
if not path_fif.exists():
    os.makedirs(path_fif)

subfolders = ['control', 'mci', 'dementia']

for subfolder in subfolders:
    dest_path = path_fif / subfolder
    if not dest_path.exists():
        os.makedirs(dest_path)


def create_raw(raw_fieldtrip):
    raw_copy = raw_fieldtrip.pick_types(meg=True)
    raw_copy.load_data()

    data = raw_copy.get_data()
    data *= 1e-15  # fT to T

    ch_types = raw_copy.get_channel_types()
    ch_names = raw_copy.ch_names
    info = mne.create_info(
        ch_names=ch_names, ch_types=ch_types, sfreq=raw_fieldtrip.info['sfreq'])

    raw_fif = mne.io.RawArray(data, info)
    return raw_fif


# Parcours des dossiers et conversion des fichiers
for subfolder in subfolders:
    folder_path = path_fieldtrip / subfolder
    for file in os.listdir(folder_path):
        if file.endswith(".mat"):
            mat_path = folder_path / file
            raw_fieldtrip = mne.io.read_raw_fieldtrip(
                mat_path, None, data_name='D')
            raw_fif = create_raw(raw_fieldtrip)
            fif_path = path_fif / subfolder / (file.replace(".mat", ".fif"))
            raw_fif.save(fif_path)

C:\Users\julie\AppData\Local\Temp\ipykernel_772\90064928.py:43: RuntimeWarning: Importing FieldTrip data without an info dict from the original file. Channel locations, orientations and types will be incorrect. The imported data cannot be used for source analysis, channel interpolation etc.
  raw_fieldtrip = mne.io.read_raw_fieldtrip(
C:\Users\julie\AppData\Local\Temp\ipykernel_772\90064928.py:43: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw_fieldtrip = mne.io.read_raw_fieldtrip(
C:\Users\julie\AppData\Local\Temp\ipykernel_772\90064928.py:43: RuntimeWarning: Cannot guess the correct type of channel AG001. Making it a MISC channel.
  raw_fieldtrip = mne.io.read_raw_fieldtrip(
C:\Users\julie\AppData\Local\Temp\ipykernel_772\90064928.py:43: RuntimeWarning: Cannot guess the correct type of channel AG002. Making it a MISC channel.
  raw_fieldtrip = mne.io.read_raw_

RuntimeError: The data you are trying to load does not seem to be raw data

In [5]:
import scipy.io
from pathlib import Path
import os

# Chemin du dossier contenant les fichiers .mat
path_to_mat_files = Path("./training")

# Dictionnaires pour stocker les données
meg_data = {
    'type': [],
    'Nsamples': [],
    'Fsample': [],
    'trial_info': [],
    'channel_info': [],
    'data': [],
    'sensor_info': []
}

# Fonction pour charger les données d'un fichier .mat
file_path = "./training/control/hokuto_control1.mat"

# Charger le fichier .mat
mat = scipy.io.loadmat(file_path)

## Extraire les champs intéressants
#meg_data['Nsamples'].append(mat['Nsamples'][0][0])
#meg_data['Fsample'].append(mat['Fsample'][0][0])
#meg_data['trial_info'].append(mat['trials'][0])
#meg_data['channel_info'].append(mat['channels'][0])
#meg_data['data'].append(mat['data'][0])
#meg_data['sensor_info'].append(mat['sensors'][0])



KeyError: 'Nsamples'